## K-Nearest Neighbors (KNN)

## Introducción

En este proyecto se aplica el algoritmo de clasificación K-Nearest Neighbors (KNN) para predecir la veracidad de afirmaciones (statements) basadas en información textual y categórica asociada, como el tema, el autor, su ocupación y afiliación política.

El enfoque combina técnicas de procesamiento de texto (TF-IDF) con codificación categórica para construir un conjunto de características robusto. Se utiliza validación cruzada y búsqueda de hiperparámetros para optimizar el modelo.


## 1. Carga de librerías
Se importan las bibliotecas necesarias para manipulación de datos, vectorización de texto, codificación de variables categóricas, entrenamiento del modelo y evaluación de resultados.

In [30]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import numpy as np


## 2. Carga y exploración del dataset

In [ ]:
# Cargar los datos
train_df = pd.read_csv('../../../data/processed/train_preprocess_v1.csv')
test_df = pd.read_csv('../../../data/processed/test_preprocess_v1.csv')

# Preprocesamiento: seleccionar las columnas relevantes
X_train_full = train_df[['statement', 'subject', 'speaker', 'speaker_job', 'party_affiliation', 
                         'party_affiliation_uni', 'party_affiliation_category_map']]
y_train = train_df['label']

X_test_full = test_df[['statement', 'subject', 'speaker', 'speaker_job', 'party_affiliation', 
                       'party_affiliation_uni', 'party_affiliation_category_map']]

# Vectorización de los textos con TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)  # Limitar el número de características para reducir la dimensionalidad
X_train_tfidf = vectorizer.fit_transform(X_train_full['statement'])
X_test_tfidf = vectorizer.transform(X_test_full['statement'])

# Para las características no textuales, usamos OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')  # Manejar categorías desconocidas con 'ignore'
X_train_additional = encoder.fit_transform(X_train_full[['subject', 'speaker', 'speaker_job', 
                                                        'party_affiliation', 'party_affiliation_uni', 
                                                        'party_affiliation_category_map']])
X_test_additional = encoder.transform(X_test_full[['subject', 'speaker', 'speaker_job', 
                                                   'party_affiliation', 'party_affiliation_uni', 
                                                   'party_affiliation_category_map']])

# Concatenar las características textuales y no textuales
X_train_final = np.hstack([X_train_tfidf.toarray(), X_train_additional])
X_test_final = np.hstack([X_test_tfidf.toarray(), X_test_additional])

# Dividir el conjunto de datos de entrenamiento en entrenamiento y validación (80/20)
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_final, y_train, test_size=0.2, random_state=42)


## 3. Entrenamiento del modelo KNN

Se entrena un clasificador KNN utilizando validación cruzada (3-fold) y búsqueda en malla (GridSearchCV) para encontrar los mejores hiperparámetros: número de vecinos (`n_neighbors`), métrica de distancia y tipo de ponderación.



In [ ]:
# Definir el clasificador KNN y los parámetros para GridSearch
knn = KNeighborsClassifier()

param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'metric': ['euclidean', 'manhattan'],
    'weights': ['uniform', 'distance']
}

# Realizar la búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(knn, param_grid, cv=3, scoring='accuracy', verbose=10, n_jobs=-1)
grid_search.fit(X_train_split, y_train_split)

# Mostrar el mejor conjunto de parámetros
print(f"Mejores parámetros encontrados: {grid_search.best_params_}")

# Usar el mejor modelo para hacer predicciones
best_knn = grid_search.best_estimator_

# Predicciones en el conjunto de validación
y_val_pred = best_knn.predict(X_val_split)



Fitting 3 folds for each of 20 candidates, totalling 60 fits


## 4. Evaluación del modelo

Con el mejor modelo encontrado por GridSearchCV, se hacen predicciones sobre el conjunto de validación. Se imprime un reporte de clasificación con precisión, recall y F1-score para cada clase.


In [ ]:
# Mostrar el reporte de clasificación
print("Classification Report para Validación:")
print(classification_report(y_val_split, y_val_pred))

## 5. Generación de predicciones para test

Finalmente, se generan predicciones sobre el conjunto de prueba utilizando el mejor modelo entrenado. 


In [ ]:
# Predicciones en el conjunto de test
y_test_pred = best_knn.predict(X_test_final)

## 6. Exportar predicciones
Los resultados se guardan en un archivo `.csv` para su envío o análisis posterior.

In [ ]:


# Guardar las predicciones en el formato adecuado
predictions = pd.DataFrame({
    'id': test_df['id'],
    'label': y_test_pred
})

predictions.to_csv("prediction27.csv", index=False)
print("Predicciones guardadas en 'predictions.csv'")

## Conclusiones

Para entrenar el modelo KNN, se definió un clasificador y se realizó una búsqueda exhaustiva de hiperparámetros utilizando GridSearchCV. Se exploraron distintas combinaciones de número de vecinos (3, 5, 7, 9, 11), métricas de distancia (euclidiana y Manhattan) y métodos de ponderación (uniforme y por distancia), con una validación cruzada de 3 folds. Este enfoque permitió identificar el mejor conjunto de parámetros basado en la métrica de exactitud, resultando en una configuración óptima con 11 vecinos, distancia euclidiana y ponderación por distancia.

El rendimiento del modelo en el conjunto de validación mostró una exactitud global del 64%, con un desempeño desigual entre las clases. La clase 1 fue detectada con buena precisión y recall, lo que indica que el modelo clasifica correctamente la mayoría de sus casos. En contraste, la clase 0 presentó una precisión y recall bajos, reflejando dificultades para identificarla adecuadamente. Esto sugiere un posible desequilibrio en las clases o diferencias en la complejidad para clasificarlas, afectando la capacidad del modelo para generalizar en la clase menos representada o más compleja.

Como próximos pasos, sería recomendable explorar técnicas para mejorar la detección de la clase con menor desempeño, tales como ajustar pesos para equilibrar las clases o aplicar métodos de sobremuestreo. Además, probar otros valores de vecinos y métricas distintas podría ayudar a optimizar aún más el modelo. Finalmente, considerar preprocesamientos adicionales o modelos alternativos puede ser útil para superar las limitaciones actuales y lograr un mejor balance en la clasificación.